In [1]:
# Install the relevant libraries
!pip install tfx tensorflow_transform python-snappy pandas requests zipfile36 pyarrow apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Import the libraries required for the example
import pandas as pd
import requests
from zipfile import ZipFile
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer
from tfx.orchestration.experimental.interactive import interactive_context
from tfx.types import Channel
import os
import shutil
import tensorflow_data_validation as tfdv
from tfx.utils import io_utils

In [8]:
# Create a funtion to import the IDEAL Household Energy metadata and survey data, call the function when required

# the web url for the data
url = "https://datashare.ed.ac.uk/bitstream/handle/10283/3647/metadata_and_surveys.zip?sequence=20&isAllowed=y"


def get_metadata_survey(url):
    file = requests.get(url, allow_redirects=True)
    content = open('metadata_and_surveys.zip', 'wb').write(file.content)
    #print(content)
    #return content

# call the function and pass the url of interest to download the data
get_metadata_survey(url)

In [4]:
# unzip the downloaded file - using Colab, file download in /content
def uzip_metadata_survey():
  with ZipFile('/content/metadata_and_surveys.zip') as mszipcontent:
        mszipcontent.extractall()

uzip_metadata_survey()

In [9]:
# check files in the parent directory
os.listdir()

['.config', 'surveys', 'metadata', 'metadata_and_surveys.zip', 'sample_data']

In [5]:
# assign the metadata directory to a variable
csv_path = '/content/metadata/'

# get a list of csv files in the folder
csv_list = [csv for csv in os.listdir(csv_path) if csv.endswith('.csv')]
for each_csv in csv_list:
    print(each_csv)

room.csv
sensor.csv
location.csv
meterreading.csv
tariff.csv
weatherfeed.csv
sensorbox.csv
home.csv
person.csv
appliance.csv
other_appliance.csv


In [10]:
# preview an example of the data, the appliance data to be used
meterreading_path = '/content/metadata/meterreading.csv'
meterreading = pd.read_csv(meterreading_path)
meterreading.head()

,homeid,provenance,provenancedetail,energytype,date,reading
0,77,technician,repair_visit,electricity,2018-01-16,30561.0
1,77,technician,repair_visit,gas,2018-01-16,8081.0
2,79,technician,repair_visit,electricity,2018-01-16,28822.0
3,79,technician,repair_visit,gas,2018-01-16,5152.0
4,96,technician,repair_visit,electricity,2018-01-15,18532.0


In [11]:
# create a pipeline directory to store pipeline outputs
# and create an interactive context
pipeline_root = '/content/pipelineroot'

IDEAL_context = interactive_context.InteractiveContext(pipeline_root=pipeline_root)

In [12]:
# make a new folder ideal_data and copy appliance data to the new folder
newfolder = 'ideal_data'
!mkdir -p "{csv_path}{newfolder}"

folder1 = '/content/metadata'
folder2 = '/content/metadata/ideal_data'
tocopy = 'meterreading.csv'

pathone = os.path.join(folder1, tocopy)
pathtwo = os.path.join(folder2, tocopy)

shutil.copyfile(pathone, pathtwo)

'/content/metadata/ideal_data/meterreading.csv'

In [13]:
# Use CsvExampleGen from ExampleGen to parse the csv data
# and convert it to suit the format for TFX internal processing.
# CsvExampleGen generates 2 outputs: Training and Evaluation outputs

IDEAL_explgen = CsvExampleGen(input_base = folder2)
IDEAL_context.run(IDEAL_explgen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
# use StatisticsGen to obtain some statistics of the data
IDEAL_stats_gen = StatisticsGen(examples = IDEAL_explgen.outputs['examples'])
IDEAL_context.run(IDEAL_stats_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
# the StatisticsGen produced two output files FeatureStats.pb saved within the pipelineroot of the google drive location
# the general path is /content/pipelineroot/StatisticsGen/statistics followed by a numerical value, then Split-eval/FeatureStats.pb
# assign relevant variables to these location and visualise the statistics of the files
stats_eval_dir = '/content/pipelineroot/StatisticsGen/statistics/2/Split-eval/FeatureStats.pb'
stats_eval = tfdv.load_stats_binary(stats_eval_dir)

# Use TFDV to produce the corresponding visualizations
tfdv.visualize_statistics(stats_eval)

In [17]:
# statistics for the train output
stats_train_dir = '/content/pipelineroot/StatisticsGen/statistics/2/Split-train/FeatureStats.pb'
stats_train = tfdv.load_stats_binary(stats_train_dir)

tfdv.visualize_statistics(stats_train)

In [18]:
# from the parsed cvs IDEAL_explgen obtain and visualise the schema using SchemaGen
IDEAL_schema_gen = SchemaGen(statistics = IDEAL_stats_gen.outputs['statistics'])
output = IDEAL_context.run(IDEAL_schema_gen)

In [19]:
# from the path to the generated schema, use TFDV to produce a visualizations
# the general path is /content/pipelineroot/SchemaGen/schema followed by a numerical value, then schema.pbtxt
schema_dir = '/content/pipelineroot/SchemaGen/schema/3/schema.pbtxt'
schema = tfdv.load_schema_text(schema_dir)

# Use TFDV to produce the corresponding visualizations
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'date',BYTES,required,,-
'energytype',STRING,required,,'energytype'
'provenance',BYTES,required,,-
'provenancedetail',STRING,required,,'provenancedetail'
'reading',FLOAT,required,,-
'﻿homeid',INT,required,,-


,Values
Domain,
'energytype',"'electricity', 'gas'"
'provenancedetail',"'all_inapp_meters_mid', 'all_web_end', 'installation_visit', 'repair_visit'"


In [20]:
# check for anomalies
IDEAL_expl_val = ExampleValidator(
    statistics = IDEAL_stats_gen.outputs['statistics'],
    schema = IDEAL_schema_gen.outputs['schema'])
IDEAL_context.run(IDEAL_expl_val)
IDEAL_context.show(IDEAL_expl_val.outputs['anomalies'])

For this example of transforming the IDEAL Household Energy dataset, the TFX transform component requires a custom transformation file which contains the transformation to be performed.

Some numeric and categorical features where tranformed

The transform file referenced in this example is ideal_transform.py


In [21]:
# Write the transform file
%%writefile ideal_transform.py

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
    # clean feature names to resolve '\ufeff' errors, this example feature_clean ignores/delete ti
    feature_clean = {key.strip('\ufeff'): value for key, value in inputs.items()}

    # Assign feature keys with the cleansed feature names
    homeid = feature_clean['homeid']
    reading = feature_clean['reading']
    date = feature_clean['date']
    energytype = feature_clean['energytype']
    provenance = feature_clean['provenance']
    provenancedetail = feature_clean['provenancedetail']

    outputs = {
        'homeid': homeid,
        'reading': reading,
        'date': date,
        'energytype': energytype,
        'provenance': provenance,
        'provenancedetail': provenancedetail
    }

    # collate features of interest that would be transformed
    NUMERIC_FEATURES_KEYS = {'reading': reading}
    CATEGORICAL_FEATURES_KEYS = {'energytype': energytype, 'provenancedetail': provenancedetail}

    for key in NUMERIC_FEATURES_KEYS:
        scaled = tft.scale_to_0_1(inputs[key]) # define a transformation to apply
        outputs[key] = tf.reshape(scaled, [-1]) # apply the transformation and reshape the data

    for key in CATEGORICAL_FEATURES_KEYS:
        outputs[key] = tft.compute_and_apply_vocabulary(inputs[key], vocab_filename=None, num_oov_buckets=0) # assign index to distinct entries. Least index for the most occuring entries.

    return outputs


Writing ideal_transform.py


In [22]:
# Transform: Data Integration and Transformation
transform = Transform(
    examples=IDEAL_explgen.outputs['examples'],
    schema = IDEAL_schema_gen.outputs['schema'],
    module_file='/content/ideal_transform.py' # the transform file could be called from an external directory
    )
IDEAL_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
# reveal the outputs of the transformed process
testing_tsl = transform.outputs['transform_graph'].get()[0].uri
os.listdir(testing_tsl)

['transform_fn', 'metadata', 'transformed_metadata']

In [86]:
# Script to extract the transformed data into a dataframe
# The location of the transformed file, the TFRecord files is in uri

# Function to convert the TFX transformed TFRecord output to a Pandas dataframe
def tfx_tfn_df(uri, ordered_features):
  # Create an empty list required to store data after extraction
  tfxdata = []

  # List the transformed files, these files are compressed as gzip
  TFRecord_files = [os.path.join(uri, name) for name in os.listdir(uri)]

  # Read these files into a record dataset TFRecordDataset
  TFR_dataset = tf.data.TFRecordDataset(TFRecord_files, compression_type="GZIP")


  # Iterate over the TFRecordDataset and interprete the gzip records
  for a_record in TFR_dataset:  #.take(4):# interest in 4 records
      numpy_output = a_record.numpy() # convert the tensor object into a numpy.ndarray object
      example_output = tf.train.Example()
      example_output.ParseFromString(numpy_output) # convert the data into an Example object.

      # Extract the features from the example
      features = example_output.features.feature
      TFRecord_data = {} # initialize an empty dictionary to store the records

      # Iterate over the desired order of features
      for key in ordered_features:
          # Checking if the feature exists in the example
          if key in features:
              # If the feature exists, checking if there are values in the features, and datatypes
              if features[key].HasField('bytes_list') and features[key].bytes_list.value:
                  TFRecord_data[key] = features[key].bytes_list.value[0].decode('utf-8')
              elif features[key].HasField('int64_list') and features[key].int64_list.value:
                  TFRecord_data[key] = features[key].int64_list.value[0]
              elif features[key].HasField('float_list') and features[key].float_list.value:
                  TFRecord_data[key] = features[key].float_list.value[0]
              elif features[key].HasField('string_list') and features[key].string_list.value:
                  TFRecord_data[key] = features[key].string_list.value[0]
              elif features[key].HasField('enum_list') and features[key].enum_list.value:
                  TFRecord_data[key] = features[key].enum_list.value[0]
          else:
              # If the value doesn't exist in the feature, then set the value to None
              TFRecord_data[key] = None

      # Append the extracted data to the dictionary
      tfxdata.append(TFRecord_data)

  # Create a DataFrame from the list of extracted data
  tfxdata = pd.DataFrame(tfxdata)

  return tfxdata # return the dataframe when this function is called

  # Display the DataFrame
  # print(tfxdata)


In [92]:
# Features are place in an order, to help compare with the original raw dataframe
ordered_features = ['homeid','provenance','provenancedetail','energytype','date','reading']

In [96]:
# Extract the records in the train sample in Split-train
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
get_train_dataframe = tfx_tfn_df(train_uri, ordered_features)

In [97]:
print(get_train_dataframe.head(4))

   homeid  provenance  provenancedetail  energytype        date   reading
0      77  technician                 3           0  2018-01-16  0.000341
1      79  technician                 3           1  2018-01-16  0.001245
2      79  technician                 3           0  2018-01-16  0.000213
3      96  technician                 3           1  2018-01-15  0.000796


In [98]:
# Extract the records in the evaluation sample in Split-eval
eval_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-eval')
get_eval_dataframe = tfx_tfn_df(eval_uri, ordered_features)

In [100]:
print(get_eval_dataframe.head(1))

   homeid  provenance  provenancedetail  energytype        date   reading
0      77  technician                 3           1  2018-01-16  0.001321


In [101]:
# the raw or original dataframe dataset
meterreading.head(5)

,homeid,provenance,provenancedetail,energytype,date,reading
0,77,technician,repair_visit,electricity,2018-01-16,30561.0
1,77,technician,repair_visit,gas,2018-01-16,8081.0
2,79,technician,repair_visit,electricity,2018-01-16,28822.0
3,79,technician,repair_visit,gas,2018-01-16,5152.0
4,96,technician,repair_visit,electricity,2018-01-15,18532.0
